In [2]:
from termcolor import colored
from selenium import webdriver
import time
import json
from collections import Counter
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinter import Tk, filedialog

def create_window(message):
    """Cria uma janela tkinter com a mensagem passada."""
    window = tk.Tk()
    window.title("Script Progresso")
    label = tk.Label(window, text=message, font=("Arial", 14))
    label.pack(pady=20)
    window.wait_window()

def load_env_variables():
    load_dotenv()
    username = r"trlegalus\taylor.teixeira"
    password = "@Rewmond222"

    return username, password


def time_trade(driver):
    username = "megan.halverson@thomsonreuters.com"
    tpassword = "Findlaw123!"

    driver.get("https://my.timetrade.com/td/login.jsp")
    user = driver.find_element("xpath", '//*[@id="email"]')
    user.send_keys(username)
    password = driver.find_element("xpath", '//*[@id="password"]')
    password.send_keys(tpassword)
    time.sleep(0.1)
    login_button = driver.find_element("xpath", '//*[@id="btnSignIn"]')
    login_button.click()
    driver.implicitly_wait(10)
    report_button = driver.find_element(
        "xpath", "//*[@id='reportMenuItem_text']")
    report_button.click()
    date_field = driver.find_element(
        "xpath", '//*[@id="dijit_layout_ContentPane_2"]/div[1]/img[1]')
    date_field.click()


def authenticate(username, password):
    auth = HTTPBasicAuth(username, password)
    return auth


def files():
    def upload_file():
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename()
        return file_path

    # Fazer upload do primeiro arquivo (Excel)
    print("Upload do arquivo Excel:")
    arquivo_xlsx = "Running Time Trade Check-1.31.23.xlsx"
    baseFrame = pd.read_excel(arquivo_xlsx)

    # Fazer upload do segundo arquivo (CSV)
    print("Upload do arquivo CSV:")
    nome_arquivo2 = upload_file()
    dataFrame = pd.read_csv(nome_arquivo2)

    # Comparar DataFrames e identificar e-mails únicos em cada um:
    unique_emails_in_uri = dataFrame[~dataFrame["Email"].isin(baseFrame["Email"])]
    return unique_emails_in_uri


def verify_contacts(auth, unique_emails_in_uri, base_url):
    # Encontre as linhas onde a coluna 'Company' está em branco
    filtered_rows = unique_emails_in_uri.loc[
        (unique_emails_in_uri['Company'].isnull()) &
        (~unique_emails_in_uri['Email'].str.contains('@thomsonreuters.com')) &
        (~unique_emails_in_uri['Email'].str.contains('@tr.com'))
        ]

    # Itere pelas linhas com 'Company' em branco
    for index, row in filtered_rows.iterrows():
        Email = row['Email']

        # Faça a chamada à API Eloqua para procurar o contato com base no email.
        dominio = Email.split('@')[1]
        search_url = f"{base_url}/api/REST/1.0/data/contacts?search=name='*@{dominio}'&depth=complete"
        response = requests.get(search_url, auth=auth)

        # Verifique se a solicitação foi bem-sucedida.
        if response.status_code == 200:
            # Analise a resposta JSON.
            data = response.json()
            if "elements" in data:
                elements = data["elements"]
                if elements:
                    account_name_found = False
                    for element in elements:
                        if "salesPerson" in element:
                            account_name = element.get("accountName")
                            account_name_found = True
                            break  # Sair do loop após encontrar a primeira correspondência

                    if not account_name_found:
                        # Se não houver salesPerson, encontre o valor de accountName mais comum.
                        account_names = [element.get("accountName") for element in elements]
                        most_common_account_name = Counter(account_names).most_common(1)[0][0]
                        print(f"O accountName mais comum para o email '{Email}' é: {most_common_account_name}")
                        search_account_name = most_common_account_name

                    else:
                        # Se houver salesPerson, encontre o valor de accountName mais comum entre os contatos com salesPerson.
                        account_names_with_salesperson = [element.get("accountName") for element in elements if
                                                          "salesPerson" in element]
                        most_common_account_name_with_salesperson = \
                        Counter(account_names_with_salesperson).most_common(1)[0][0]
                        print(
                            f"O accountName mais comum entre os contatos com salesPerson para o email '{Email}' é: {most_common_account_name_with_salesperson}")
                        search_account_name = most_common_account_name_with_salesperson

                    # Atualize o valor na planilha unique_emails_in_uri na mesma linha.
                    if search_account_name is not None and search_account_name != "GA PUBLIC DEFENDER":
                        if "&amp;" in search_account_name:
                            search_account_name = search_account_name.replace("&amp;", "&")

                        unique_emails_in_uri.at[index, 'Company'] = search_account_name

                    # print(f"O valor na linha {index} da coluna 'Company' na planilha 'unique_emails_in_uri' foi atualizado para: {search_account_name}")

    print("Inicializando upload para o Eloqua...")

    # Filtrar linhas onde 'Company' não está em branco
    unique_emails_in_uri_cleaned = unique_emails_in_uri.dropna(subset=['Company'])

    # Abrir o arquivo Excel existente como um DataFrame
    try:
        dados_existente = pd.read_excel(
            'Running Time Trade Check-1.31.23.xlsx', sheet_name='Report_7D3203233C9FB104B6C361BD')
    except FileNotFoundError:
        dados_existente = pd.DataFrame()

    # Concatenar os dados existentes com os dados novos
    dados_concatenados = pd.concat(
        [dados_existente, unique_emails_in_uri_cleaned], ignore_index=True)

    # Escrever os dados concatenados de volta ao arquivo Excel
    with pd.ExcelWriter('./Running Time Trade Check-1.31.23.xlsx', engine='openpyxl') as writer:
        dados_concatenados.to_excel(
            writer, sheet_name='Report_7D3203233C9FB104B6C361BD', index=False)

    unique_emails_in_uri_cleaned.to_excel('emails.xlsx', index=False)

    # URL da API para fazer uma solicitação POST
    post_url = f"{base_url}/api/REST/2.0/data/customObject/376/instance"

    # Estrutura JSON para o corpo da solicitação
    json_data = {
        "type": "CustomObjectData",
        "id": "212502915",
        "mapDataCards": "true",
        "fieldValues": [
            {
                "type": "FieldValue",
                "id": "5917",
                "value": "Email"
            },
            # {
            #     "type": "FieldValue",
            #     "id": "17337",
            #     "value": "Appointment Time (Date/Time Format)"
            # },
            {
                "type": "FieldValue",
                "id": "5919",
                "value": "Appointment Time"
            },
            {
                "type": "FieldValue",
                "id": "5922",
                "value": "Organizer Email"
            },
            {
                "type": "FieldValue",
                "id": "5924",
                "value": "Business Phone"
            },
            {
                "type": "FieldValue",
                "id": "5925",
                "value": "Company"
            },
            {
                "type": "FieldValue",
                "id": "5928",
                "value": "Confirmation Number"
            },
            {
                "type": "FieldValue",
                "id": "5935",
                "value": "Invitee Comment"
            },
            {
                "type": "FieldValue",
                "id": "5932",
                "value": "Invitee Response"
            },
            {
                "type": "FieldValue",
                "id": "5930",
                "value": "Organizer Question"
            },
            {
                "type": "FieldValue",
                "id": "5926",
                "value": "Activity Name"
            },
            {
                "type": "FieldValue",
                "id": "5923",
                "value": "First and Last Name"
            },
            {
                "type": "FieldValue",
                "id": "5921",
                "value": "Organizer Name"
            },
            {
                "type": "FieldValue",
                "id": "5920",
                "value": "Duration"
            }
            # {
            #     "type": "FieldValue",
            #     "id": "12178",
            #     "value": "SFDC GLI Lead ID"
            # },
            # {
            #     "type": "FieldValue",
            #     "id": "18386",
            #     "value": "Record Create Date"
            # }
        ]
    }

    # Mapear os valores do JSON para os nomes da planilha
    id_to_value_mapping = {
        "5917": "Email",
        # "17337": "Appointment Time (Date/Time Format)",
        "5919": "Appointment Time",
        "5922": "Organizer Email",
        "5924": "Phone Number",
        "5925": "Company",
        "5928": "Confirmation Number",
        "5935": "Invitee Comment",
        "5932": "Invitee Response",
        "5930": "Organizer Question",
        "5926": "Activity Name",
        "5923": "Invitee Name",
        "5921": "Organizer Name",
        "5920": "Duration",
        # "12178": "SFDC GLI Lead ID",
        # "18386": "Record Create Date"
    }

    # Iterar sobre todos os contatos da planilha
    for index, row in unique_emails_in_uri_cleaned.iterrows():
        # Para cada contato, crie uma cópia do JSON original
        updated_json_data = json_data.copy()

        # Iterar sobre as linhas do JSON e atualizar os valores com base na planilha
        for field_value in updated_json_data["fieldValues"]:
            field_id = str(field_value["id"])
            if field_id in id_to_value_mapping:
                value = id_to_value_mapping[field_id]
                # Procurar na planilha o nome relacionado ao valor do JSON
                if value in unique_emails_in_uri_cleaned.columns:
                    new_value = row[value]
                    # Se o valor na planilha estiver vazio, defina o valor no JSON como uma string vazia
                    if pd.isna(new_value):
                        new_value = ""
                    field_value["value"] = new_value

        # Enviar a solicitação POST com o JSON atualizado para o contato atual
        response = requests.post(post_url, auth=auth, json=updated_json_data)
        if response.status_code == 201:
            print(f"Solicitação POST bem-sucedida para o contato {index}.")
        else:
            print(f"Erro para o contato {index}: {response.status_code}")

    return unique_emails_in_uri_cleaned


# ------------------------------------------------------------------------------

def update_lookup(c_email, auth):
    def get_contact_id(c_email, auth):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contacts?search=name="{c_email}"&depth=complete'
        response = requests.get(url, auth=auth)

        if response.status_code == 200:
            contacts = response.json()
            if contacts and 'elements' in contacts and contacts['elements']:
                return contacts['elements'][0]['id'], contacts
            else:
                return None, None

    def update_contact(contact_id, c_email, nome_do_campo_value, auth):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
        payload = {
            "type": "Contact",
            "id": contact_id,
            "emailAddress": c_email,
            "salesPerson": nome_do_campo_value
        }

        response = requests.put(url, json=payload, auth=auth)

        if response.status_code == 200:
            print('Informações do contato atualizadas com sucesso.')

    nome_lookup_table = "FindLaw Rep to user ID"

    # Passo 1: Fazer solicitação à primeira API
    contact_id, contact_data = get_contact_id(c_email, auth)
    if contact_data and "elements" in contact_data:
        # Iterar sobre os elementos (contatos)
        for contact in contact_data.get("elements", []):
            # Extrair C_Salesperson do JSON
            c_salesperson = contact.get("salesPerson", None)
            c_email = contact.get("emailAddress", None)

            if c_salesperson is not None and not ' ' in c_salesperson:
                return True

            # Verificar se C_Salesperson é nulo ou vazio
            if c_salesperson is None or not c_salesperson.strip():
                print("Passando para o próximo objeto.")
                continue

            search = c_salesperson
            search = search.replace(' ', '*')

            # Pesquisar na segunda API para obter o primeiro objeto dentro de fieldValues
            salesperson_api_url = f"https://secure.p04.eloqua.com/api/REST/1.0/data/customObject/1143?search=name='{search}*'"
            response = requests.get(salesperson_api_url, auth=auth)
            salesperson_data = response.json()

            # Verificar se a solicitação foi bem-sucedida
            if response.status_code != 200:
                print(
                    f"Falha ao obter dados do salesperson. Código de status: {response.status_code}")
                print(f"Resposta: {response.text}")
                continue

            # Extrair informações do primeiro objeto dentro de fieldValues
            if "elements" in salesperson_data and salesperson_data["elements"]:
                field_values = salesperson_data["elements"][0].get("fieldValues", [])
                if not field_values:
                    print("Campo fieldValues não encontrado no JSON do salesperson.")
                    continue
            else:
                print("Lista de elementos vazia ou chave 'elements' não encontrada no JSON do salesperson.")
                continue

            # esse é o ID do lookup
            nome_do_campo_value = field_values[0]["value"]
            # Concatenando valores do terceiro e quarto campos
            nome_do_campo_key = f"{field_values[2]['value']} {field_values[3]['value']}"

            # Passo 1: Buscar o ID da lookup table
            eloqua_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTables?search=name='{nome_lookup_table}'"
            response = requests.get(eloqua_url, auth=(auth))
            lookup_table_data = response.json()
            lookup_table_id = lookup_table_data["elements"][0]["id"]
            print(f"ID da lookup table {nome_lookup_table}: {lookup_table_id}")

            lookup_table_entries_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTable/{lookup_table_id}/lookupTableEntries"
            page = 1
            total_pages = 1  # Inicializado com 1 para entrar no loop

            key_found = False  # Variável para indicar se a chave foi encontrada

            while page <= total_pages and not key_found:
                # Adiciona a lógica de paginação
                query_params = {"page": page}

                response = requests.get(
                    lookup_table_entries_url, params=query_params, auth=(auth))

                if response.status_code == 200:
                    entries_data = response.json()

                    # Processa as entradas na página atual
                    for entry in entries_data["elements"]:
                        key = entry.get("key")
                        if key == c_salesperson:
                            print(
                                f"A chave {c_salesperson} já existe na lookup table. O script não adicionará um novo valor.")
                            # Passo 1: Buscar o ID do contato
                            contact_id = get_contact_id(c_email, auth)

                            if contact_id:
                                # Passo 2: Atualizar as informações do contato
                                update_contact(contact_id, c_email,
                                               nome_do_campo_value, auth)
                            else:
                                print('Contato não encontrado.')
                            key_found = True  # Define a variável para indicar que a chave foi encontrada
                            break  # Pode interromper o loop se a chave for encontrada

                    # Atualiza informações de paginação
                    total_pages = (entries_data.get("total") //
                                   entries_data.get("pageSize")) + 1
                    page += 1

            # Passo 2: Adicionar um novo valor à lookup table, se a chave não foi encontrada
            if not key_found:
                lookup_table_entry_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTable/{lookup_table_id}/lookupTableEntry"
                lookup_table_entry_data = {
                    "type": "LookupTableEntry",
                    "key": f"{c_salesperson}",
                    "value": f"{nome_do_campo_value}"
                }

                headers = {
                    "Content-Type": "application/json"
                }

                response = requests.post(lookup_table_entry_url, data=json.dumps(
                    lookup_table_entry_data), headers=headers, auth=(auth))

                # Verificar se a operação foi bem-sucedida
                if response.status_code == 201:
                    print(
                        f"Novo valor adicionado com sucesso à lookup table {nome_lookup_table}")
                    # Passo 1: Buscar o ID do contato
                    contact_id = get_contact_id(c_email, auth)

                    # Pesquisar na segunda API para obter o primeiro objeto dentro de fieldValues
                    salesperson_api_url = f"https://secure.p04.eloqua.com/api/REST/1.0/data/customObject/1143?search=name='{search}*'"
                    response = requests.get(salesperson_api_url, auth=auth)
                    salesperson_data = response.json()

                    # Verificar se a solicitação foi bem-sucedida
                    if response.status_code != 200:
                        print(
                            f"Falha ao obter dados do salesperson. Código de status: {response.status_code}")
                        print(f"Resposta: {response.text}")
                        continue

                    # Extrair informações do primeiro objeto dentro de fieldValues
                    if "elements" in salesperson_data and salesperson_data["elements"]:
                        field_values = salesperson_data["elements"][0].get("fieldValues", [])
                        if not field_values:
                            print("Campo fieldValues não encontrado no JSON do salesperson.")
                            continue
                    else:
                        print("Lista de elementos vazia ou chave 'elements' não encontrada no JSON do salesperson.")
                        continue

                    # esse é o ID do lookup
                    nome_do_campo_value = field_values[0]["value"]

                    if contact_id:
                        # Passo 2: Atualizar as informações do contato
                        update_contact(contact_id, c_email,
                                       nome_do_campo_value, auth)
                    else:
                        print('Contato não encontrado.')
                else:
                    print(
                        f"Erro ao adicionar valor à lookup table. Código de status: {response.status_code}")
                    print(f"Resposta: {response.text}")


def update_country_eloqua(email, auth):
    # Etapa 1: Pesquisar por email
    search_url = f'https://secure.p04.eloqua.com/api/REST/2.0/data/contacts?search={email}'
    response = requests.get(search_url, auth=auth, headers={
        'Content-Type': 'application/json'})

    # Verificar se a resposta foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Obter o ID do primeiro objeto retornado
        contacts = response.json().get('elements', [])
        if contacts:
            contact_id = contacts[0]['id']

            # Etapa 2: Pesquisar por ID
            get_url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
            response = requests.get(get_url, auth=auth, headers={
                'Content-Type': 'application/json'})

            if response.status_code == 200:
                # Etapa 3: Atualizar o país para "US"
                put_url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
                payload = {
                    "type": "Contact",
                    "emailAddress": email,
                    "id": contact_id,
                    "country": "US"
                }
                response = requests.put(put_url, json=payload, auth=auth, headers={
                    'Content-Type': 'application/json'})

                if response.status_code == 200:
                    return f"O país para o contato com ID {contact_id} foi atualizado para 'US'."
                else:
                    return f"Erro ao atualizar o país: {response.status_code}"
        else:
            return "Nenhum contato encontrado com o email fornecido."


def eloqua_api_update_contact_field(email, most_common_value_100100, auth):
    # Função para buscar o ID do Contato pelo E-mail
    def get_contact_id(email):
        url = f'https://secure.p04.eloqua.com/api/REST/2.0/data/contacts?search={email}'
        response = requests.get(url, auth=auth)

        if response.status_code == 200:
            contact_data = response.json().get('elements', [])
            if contact_data:
                return contact_data[0]['id']
        return None

    # Função para realizar a Operação de PUT
    def update_contact_field(contact_id, most_common_value_100100, email):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
        headers = {'Content-Type': 'application/json'}
        data = {
            "type": "Contact",
            "id": contact_id,
            "emailAddress": email,
            "fieldValues": [
                {
                    "type": "FieldValue",
                    "id": "100317",
                    "value": most_common_value_100100
                }
            ]
        }
        response = requests.put(url, json=data, headers=headers, auth=auth)
        return response.status_code

    # Exemplo de uso
    contact_id = get_contact_id(email)

    if contact_id:
        status_code = update_contact_field(
            contact_id, most_common_value_100100, email)

        if not status_code == 200:
            print(
                f'Erro ao atualizar o contato. Código de status: {status_code}')
    else:
        print(f'Contato com o e-mail {email} não encontrado.')


def search_contact_by_email(base_url, auth, email):
    # dominio = email.split('@')[1]
    search_url = f"{base_url}/api/REST/1.0/data/contacts?search=name={email}&depth=complete"
    response = requests.get(search_url, auth=auth)
    return response


def verify_us_country(email, base_url, auth):
    response = search_contact_by_email(base_url, auth, email)
    if response.status_code == 200:
        data = response.json()
        if "elements" in data:
            elements = data["elements"]
            for element in elements:
                if "country" in element and element["country"] == "US":
                    return True

                # Coletar os valores do campo "country" de todos os contatos
                countries = [element.get(
                    "country") for element in elements if "country" in element]
                total_contacts = len(countries)
                us_count = countries.count("US")

                if total_contacts > 0:
                    if us_count / total_contacts >= 0.5:
                        print(colored(
                            f"Email: {email} - A maioria dos contatos tem 'country' igual a 'US'.", "green"))
                        # Chamada da função
                        update_country_eloqua(email, auth)
                    else:
                        print(colored(
                            f"Email: {email} - A maioria dos contatos tem 'country' diferente de 'US'.", "red"))
                else:
                    print(colored(
                        f"Email: {email} - Assuming 'US'.", "green"))
                    update_country_eloqua(email, auth)
    else:
        print(f"Email: {email} - Falha na busca de contatos por e-mail.")


def search_account_by_name(base_url, auth, search_account_name):
    account_search_url = f"{base_url}/api/REST/1.0/data/accounts?search=name='*{search_account_name}*'&depth=complete"
    account_response = requests.get(account_search_url, auth=auth)
    return account_response


def find_most_common_account_name(elements, with_salesperson):
    account_names = [element.get("accountName") for element in elements if (
            with_salesperson and "salesPerson" in element) or (not with_salesperson)]
    if account_names:
        return Counter(account_names).most_common(1)[0][0]
    else:
        return None


def verify_account(email, base_url, auth):
    response = search_contact_by_email(base_url, auth, email)
    # print(f"Email: {email} - Verificando Account...")

    if response.status_code == 200:
        data = response.json()
        if "elements" in data:
            elements = data["elements"]
            if elements:
                for element in elements:
                    if "fieldValues" in element:
                        fieldValues_data = element.get("fieldValues")
                        if isinstance(fieldValues_data, list):
                            for field_data in fieldValues_data:
                                if "id" in field_data and "value" in field_data:
                                    if field_data["id"] == "100317":
                                        return True

                        account_name_found = True
                        break

                if not account_name_found:
                    print(
                        f"Email: {email} - Nenhum contato encontrado com 'salesPerson'.")
                    search_account_name = find_most_common_account_name(
                        elements, with_salesperson=False)
                    print(f"Email: {email} - Sem Sales: {search_account_name}")
                else:
                    search_account_name = find_most_common_account_name(elements, with_salesperson=True)
                    print(f"Email: {email} - com Sales: {search_account_name}")
                    account_response = None

                    if search_account_name is not None:
                        search_account_name = search_account_name.replace(",", "").replace(
                            ".", "").replace("Group", "").replace("LLC","").replace("PLLC","").replace(" ", "*").replace("™", "").replace("&", "").replace("and", "").replace(
                            "**", "*")
                        account_response = search_account_by_name(
                            base_url, auth, search_account_name)

                if account_response is not None and account_response.status_code == 200:
                    account_data = account_response.json()
                    if "elements" in account_data:
                        account_elements = account_data["elements"]
                        if account_elements:
                            values_100464 = []

                            for account in account_elements:
                                field_values = account.get("fieldValues", [])
                                for field in field_values:
                                    if field.get("id") == "100464":
                                        values_100464.append(
                                            (float(field.get("value")), field_values))

                            if values_100464:
                                max_value_100464, max_value_field_values = max(
                                    values_100464, key=lambda x: x[0])
                                value_100100 = next((field.get(
                                    "value") for field in max_value_field_values if field.get("id") == "100100"), None)

                                #  value_100100 é o ID do Account
                                if value_100100:
                                    print(colored(
                                        f"Email: {email} - O valor do campo com id 100100 que está dentro do mesmo fieldValue é: {value_100100}",
                                        "green"))
                                    most_common_value_100100 = value_100100
                                    eloqua_api_update_contact_field(
                                        email, most_common_value_100100, auth)
                                else:
                                    print(
                                        f"Email: {email} - Nenhum valor encontrado para o id 100100 no mesmo fieldValue.")
                            else:
                                values_100100 = [
                                    field.get("value") for field in field_values if field.get("id") == "100100"]
                                most_common_value_100100 = Counter(values_100100).most_common(1)[
                                    0][0] if values_100100 else None

                                if most_common_value_100100:
                                    print(colored(
                                        f"Email: {email} - O valor do campo com id 100100 que mais se repete é: {most_common_value_100100}",
                                        "green"))
                                    eloqua_api_update_contact_field(
                                        email, most_common_value_100100, auth)
                                else:
                                    print(
                                        f"Email: {email} - Nenhum valor encontrado para os ids 100464 ou 100100 na conta.")


from bs4 import BeautifulSoup

def update_address(email, base_url, auth):
    print(f'Atualizando o endereço para o contato com o e-mail {email}...')
    # Passo 1: Pesquisar o contato por e-mail
    search_url = f"{base_url}/api/REST/1.0/data/contacts?search=name={email}&depth=complete"
    contact_data = requests.get(search_url, auth=auth)
    contact_data = contact_data.json()

    if 'elements' in contact_data and contact_data['elements']:
        contact = contact_data['elements'][0]
        contact_id = contact.get('id')
        first_name = contact.get('firstName', '')
        last_name = contact.get('lastName', '')
        account_name = contact.get('accountName', '')

        # Passo 2: Ajustar o nome da conta para pesquisa
        search_account_name = account_name.replace(",", "").replace(".", "").replace("Group", "").replace("PLLC", "").replace("LLC", "").replace(" ", "+").replace("™", "").replace("&", "").replace("and", "").replace("**", "*")

        # Passo 3: Realizar web scraping para encontrar o bloco de endereço do Google
        search_query = f"{search_account_name}+address"
        search_url = f"https://www.google.com/search?q={search_query}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        address1 = ''
        city = ''

        # Encontrar o bloco de informações da empresa
        knowledge_panel = soup.find('div', {'class': 'kno-rdesc'})
        if knowledge_panel:
            address_div = knowledge_panel.find('div', {'data-attrid': 'kc:/organization/organization:address'})
            if address_div:
                address_spans = address_div.find_all('span')
                address1 = address_spans[0].text.strip() if len(address_spans) > 0 else ''
                print(f'Endereço encontrado: {address1}')
                city = address_spans[-1].text.strip() if len(address_spans) > 1 else ''

        # Passo 4: Atualizar o contato
        update_url = f'{base_url}/api/REST/1.0/data/contact/{contact_id}'
        headers = {'Content-Type': 'application/json'}
        data = {
            "type": "Contact",
            "id": contact_id,
            "emailAddress": email,
            "firstName": first_name,
            "lastName": last_name,
            "address1": address1,
            "city": city
        }
        update_response = requests.put(update_url, json=data, headers=headers, auth=auth)
        if update_response.status_code == 200:
            print(f'Contato {email} atualizado com sucesso.')
        else:
            print(f'Erro ao atualizar o contato {email}. Código de status: {update_response.status_code}')
    else:
        print(f'Contato com o e-mail {email} não encontrado.')
# ------------------------------------------------------------------------------

def get_contact_id(email, auth):
    search_url = f'https://secure.p04.eloqua.com/api/REST/2.0/data/contacts?search={email}'
    response = requests.get(search_url, auth=auth)
    if response.status_code == 200:
        contact_data = response.json().get('elements', [])
        if contact_data:
            return contact_data[0]['id']
    return None

def update_contact(contact_id, email, first_name, last_name, auth):
    update_url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
    headers = {'Content-Type': 'application/json'}
    data = {
        "type": "Contact",
        "id": contact_id,
        "emailAddress": email,
        "firstName": first_name,
        "lastName": last_name
    }
    response = requests.put(update_url, json=data, headers=headers, auth=auth)
    return response.status_code

def update_name(df, auth):
    for index, row in df.iterrows():
        email = row['Email']
        invitee_name = row['Invitee Name']
        
        if pd.notna(invitee_name):
            name_parts = invitee_name.split()
            first_name = name_parts[0]
            last_name = " ".join(name_parts[1:]) if len(name_parts) > 1 else ""
            
            contact_id = get_contact_id(email, auth)
            if contact_id:
                status_code = update_contact(contact_id, email, first_name, last_name, auth)
                if status_code == 200:
                    print(f'Contato {email} atualizado com sucesso.')
                else:
                    print(f'Erro ao atualizar o contato {email}. Código de status: {status_code}')
            else:
                print(f'Contato com o e-mail {email} não encontrado.')


def execute_process(df, emails, auth, base_url):
    # Verify address for each email
    # for email in emails:
    #     update_address(email, base_url, auth)
    # print("Endereço atualizado com sucesso")
    
    # Verify name for each email
    update_name(df, auth)
    print("Name atualizado com sucesso!")
        
    # Verify Lookup for each email
    for c_email in emails:
        update_lookup(c_email, auth)
    print("Lookup atualizado com sucesso!")

    # Verify US Country for each email
    for email in emails:
        verify_us_country(email, base_url, auth)
    print("País atualizado com sucesso!")

    # Verify Account for each email
    for email in emails:
        verify_account(email, base_url, auth)
    print("Account atualizado com sucesso!")


if __name__ == "__main__":
    base_url = "https://secure.p04.eloqua.com"
    username, password = load_env_variables()
    auth = authenticate(username, password)

    # ------------------------------------------------------------------------------
    driver = webdriver.Chrome()
    time_trade(driver)

    unique_emails_in_uri = files()
    unique_emails_in_uri_cleaned = verify_contacts(auth, unique_emails_in_uri, base_url)
    emails = unique_emails_in_uri_cleaned['Email']
    print("POST feito com sucesso!")

    create_window("Wait 20min to finish...")
    time.sleep(60 * 20)

    # --- Carregar emails do Excel ---
    df = pd.read_excel('./emails.xlsx')
    emails = df['Email']

    # ------------------------------------------------------------------------------
    execute_process(df, emails, auth, base_url)
    create_window("Finish!!")


Upload do arquivo Excel:
Upload do arquivo CSV:


In [6]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\6125974\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import requests
from bs4 import BeautifulSoup

def get_company_address(company_name):
    # Formata a URL de busca no Google (ou outro mecanismo de busca)
    search_url = f"https://www.google.com/search?q={company_name}+endereço"
    
    # Realiza a solicitação HTTP para a página de resultados de busca
    response = requests.get(search_url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # Verifica se a solicitação foi bem-sucedida
    if response.status_code != 200:
        return None
    
    # Analisa o conteúdo HTML da página
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Procura pelo elemento que contém o endereço (isso pode variar de acordo com o layout da página)
    address = None
    for div in soup.find_all("div", class_="BNeawe deIvCb AP7Wnd"):
        if "endereço" in div.text.lower():
            address = div.text
            break
    
    return address

# Exemplo de uso
company_name = "Google"
address = get_company_address(company_name)
if address:
    print(f"Endereço da empresa {company_name}: {address}")
else:
    print(f"Não foi possível encontrar o endereço da empresa {company_name}.")

Não foi possível encontrar o endereço da empresa Google.
